**MPCP Solver**

This notebook defines a solver for the MPCP problem, as defined by Sipser.  MPCP is not decidable in general, so the integer parameter B controls how long we are willing to search for a match, before giving up.

The first code cell defines the solver (a big blob of code, you don't need to study it).  The next few code cells run some examples, and the last code cell contains a language example.  For a homework exercise, you may be asked to modify *L_dominos* in the last cell.

**Note:** you may need to "Run" the first cell, before you can run (re-evaluate) the last cell.

In [0]:
# Here we define an MPCP solver, with two versions:
# 
#    mpcp_quiet(dominos, B) -- quietly returns something
#    mpcp(dominos, B)       -- prints result, returns nothing
#
# 'dominos' is a list of dominos, where a domino is a pair of strings
# (as described by Sipser,see some examples below).
# These solvers look for a match starting with the first domino.
# 'B' is an integer bounding the length of our search, you may omit
# it to use a default value.

# First, a few helper functions
def reduce(a, b):
  # Return prefix-reduced form of strings a and b, or None.
  n = min(len(a), len(b))
  return None if a[:n]!=b[:n] else (a[n:], b[n:])

def seq2list(seq):
  # Convert (3, (2, (1, None))) to [1, 2, 3]
  ret = []
  while seq != None:
    ret.append(seq[0])
    seq = seq[1]
  return ret[::-1]

import heapq
def pqpair(r):
  # Order our pq to prefer exploring partial matches
  # with a minimal amount of unmatched text.
  return (len(r[0])+len(r[1]), r)

def mpcp_quiet(dominos, B=10000):
  """
  Given a list of dominos, try to find an MPCP match.
  That is, a match starting with a copy of dominos[0].
  If found, the match is returned as a list of indices.
  The integer B bounds the number of steps searched.
  If there is definitely no match, returns None.
  If we ran out of steps, returns B.
  """
  pq, seq = [], {}
  start = reduce(dominos[0][0], dominos[0][1])
  if start:
    heapq.heappush(pq, pqpair(start))
    seq[start] = (0, None)
  goal = ('', '')
  for step in range(B):
    if len(pq)==0: # no solution!
      return None
    thispair = heapq.heappop(pq)[1]
    thisseq = seq[thispair]
    if thispair == goal: # found it! return match
      return seq2list(thisseq)
    # Try extending thispair with each domino
    for i in range(len(dominos)):
      dom = dominos[i]
      r = reduce(thispair[0]+dom[0], thispair[1]+dom[1])
      if r and (r not in seq): # new reduced pair
        seq[r] = (i, thisseq)
        if r==goal: # found it!
          return seq2list(seq[r])
        # save r for later
        heapq.heappush(pq, pqpair(r))
  # ran out of steps
  return B

def mpcp(dominos, B=10000):
  """
  Run mpcp_quiet(dominos, B), and print the result.
  This function returns nothing.
  """
  print('Solving MPCP:', 
        ' '.join("%s/%s" % d for d in dominos))
  sol = mpcp_quiet(dominos, B)
  if sol==None:
    print("NO MATCH (no solution exists)")
  elif sol==B:
    print("NO MATCH FOUND (gave up after %d steps)" % B)
  else:
    print("FOUND MATCH:", ' '.join(str(i) for i in sol))
    print(' '.join(dominos[i][0] for i in sol))
    print(' '.join(dominos[i][1] for i in sol))
  print()  # return nothing

**Examples**

In [0]:
# Example from Sipser page 227
page227 = [('b', 'ca'), ('a', 'ab'), ('ca', 'a'), ('abc', 'c')]
mpcp(page227) # no solution

# Rearraged, moved the second domino to the front
page227b = [('a', 'ab'), ('b', 'ca'), ('ca', 'a'), ('abc', 'c')]
mpcp(page227b) # finds match in book

# Exercise 5.3, Sipser page 239
page239 = [('ab', 'abab'), ('b', 'a'), ('aba', 'b'), ('aa', 'a')]
mpcp(page239)  # finds match in book

Solving MPCP: b/ca a/ab ca/a abc/c
NO MATCH (no solution exists)

Solving MPCP: a/ab b/ca ca/a abc/c
FOUND MATCH: 0 1 2 0 3
a b ca a abc
ab ca a ab c

Solving MPCP: ab/abab b/a aba/b aa/a
FOUND MATCH: 0 0 2 1 1 3 3
ab ab aba b b aa aa
abab abab b a a a a



In [0]:
# Examples with no solution
mpcp([('b', 'baaab'), ('aa', '')]) # no solution
mpcp([('b', 'baab'), ('aa', '')]) # solver gives up

Solving MPCP: b/baaab aa/
NO MATCH (no solution exists)

Solving MPCP: b/baab aa/
NO MATCH FOUND (gave up after 10000 steps)



**Language Example**

This last code cell defines a language L (see the comments below).  As given, L is the set of all strings, over alphabet {a,b}, with an odd number of b's.

As an exercise, you may be asked to modify the list *L_dominos* (below) so that  *L* becomes some other language.

Note we add a first domino ('#Sw', '#'), where w is the input string. This 'S' looks like the start state in Sipser's reduction (from A<sub>TM</sub> to MPCP).  However, you are *not* required to use Sipser's method, in particular your dominos do not need to simulate a Turing machine.

In [0]:
# TODO: edit L_dominos here, to change the language L below.

L_dominos = [
  ('a', 'a'), ('b', 'b'), ('#', '#'),
  ('Sa', 'S'), ('Sb', 'T'),
  ('Ta', 'T'), ('Tb', 'S'), ('#T#', '#')
]

# You (or a grader!) may pick w, an input string in {a,b}*
w = 'ababab'

# You may also pick B (depending on w), large enough to find a
# match if there is one. This B should suffice for small examples:
B = 10000

# NOTE 1: L_dominos should not depend on w or B.
# NOTE 2: You should not need to edit the code below!

assert all(c in 'ab' for c in w), "string w should be in {a,b}*"

def dominos(w):
  return [('#', '#S'+w+'#')] + L_dominos

# We define dominos(w) by creating a first domino containing w,
# and prepending that to L_dominos.  We define the language
#    L = { w in {a,b}*: dominos(w) has a match }.
# If such a match exists, then we can find it with our solver,
# for large enough B.

mpcp(dominos(w), B)

Solving MPCP: #/#Sababab# a/a b/b #/# Sa/S Sb/T Ta/T Tb/S #T#/#
FOUND MATCH: 0 4 2 1 2 1 2 3 5 1 2 1 2 3 6 2 1 2 3 7 1 2 3 4 2 3 5 8
# Sa b a b a b # Sb a b a b # Ta b a b # Tb a b # Sa b # Sb #T#
#Sababab# S b a b a b # T a b a b # T b a b # S a b # S b # T #

